In [1]:
from datetime import date
import holidays
import pandas as pd
pd.set_option('display.max_rows', None)
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
startyear=2024
endyear=2030

In [3]:
data=[]
countries=['BG','CO','IL','IN','JM','US','RU']
for year in range(startyear,endyear+1):
    for i in countries:
        for j,k in holidays.country_holidays(country=i,years=year,language='en_US').items():
            data.append([i,j,k])
    

df=pd.DataFrame(data)
df.columns=['country','date','holiday']
df['holiday']=np.where((df['country']=='RU') & (df['holiday']=='Christmas Day'),'Orthodox Christmas',df['holiday'])
del df['country']
df=df.drop_duplicates()
df['holiday'] = df['holiday'].replace({'Pesach': 'Start of Passover',
                'Maundy Thursday':'Holy Thursday','Easter':'Orthodox Pascha','Christmas Day':'Christmas'})


holidays_list=["New Year's Day", "Orthodox Christmas", "Ash Wednesday", "Start of Passover", "Holy Thursday",
               "Easter Sunday","Eid ul-Fitr (estimated)", "Eid al-Adha (estimated)", "Easter", "Rosh Hashanah", 
               "Yom Kippur", "Diwali", "Christmas"]

df1=df[(df['holiday']!='New Year Holidays') & (~df['holiday'].str.contains('Day off'))]
df2=df1[df1['holiday'].isin(holidays_list)].reset_index(drop=True)


In [4]:
data_scrap=[]
for year in range(startyear,endyear+1):
    url = "https://www.timeanddate.com/holidays/india/"+str(year)
    page = urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    table = soup.find('table', attrs={'id':'holidays-table'})
    for row in table.find('tbody').find_all('tr'):
        columns = row.find_all('td')
        dates=row.find_all('th')
        if(columns != []):
            day = columns[0].text.strip()
            name = columns[1].text.strip()
        if(dates != []):
            date=dates[0].text.strip()
            data_scrap.append([year,date,name])
indian_holidays=pd.DataFrame(data_scrap)
indian_holidays.columns=['year','date','holiday']
indian_holidays['date']=pd.to_datetime(indian_holidays['date'],format="%b %d")
indian_holidays['date']=indian_holidays['date'].astype(str)
indian_holidays['year']=indian_holidays['year'].astype(str)
indian_holidays['date']=indian_holidays['date'].str.replace('1900','')
indian_holidays['date']=indian_holidays['year']+indian_holidays['date']
indian_holidays['holiday'] = indian_holidays['holiday'].replace({
                'Guru Govind Singh Jayanti': "Guru Gobind Singh's Birthday",
                'Ganesh Chaturthi/Vinayaka Chaturthi':'Ganesh Chaturthi',
                'Guru Nanak Jayanti':"Guru Nanak's Birthday"})

indian_holidays['holiday'] = indian_holidays['holiday'].str.replace('Ramadan Start','Ramadan Begins')


holidays_list_india=["Guru Gobind Singh's Birthday",'Ramadan Begins','Vaisakhi','Ganesh Chaturthi',
           "Guru Nanak's Birthday","Good Friday"]

indian_holidays_1=indian_holidays[(indian_holidays['holiday'].isin(holidays_list_india)) | (indian_holidays['holiday'].str.contains('Ramadan')) ]
del indian_holidays_1['year']



        

In [5]:
#Combined both
df_holidays=pd.concat([indian_holidays_1,df2]).reset_index(drop=True)
df_holidays=df_holidays.astype(str)
df_holidays=df_holidays.sort_values('date').reset_index(drop=True)
df_holidays=df_holidays[~((df_holidays['date'].str.contains('-12-26')) & (df_holidays['holiday']=='Christmas'))]

In [6]:
adding_Vaisakhi={'date':['2026-04-14','2027-04-14','2028-04-13','2029-04-14','2030-04-14'],
                 'holiday':['Vaisakhi','Vaisakhi','Vaisakhi','Vaisakhi','Vaisakhi']}
adding_Vaisakhi=pd.DataFrame(adding_Vaisakhi)


adding_Nanak={'date':['2026-11-24','2027-11-14','2028-11-02','2029-11-21','2030-11-10'],
                 'holiday':["Guru Nanak's Birthday","Guru Nanak's Birthday","Guru Nanak's Birthday",
                           "Guru Nanak's Birthday","Guru Nanak's Birthday"]}
adding_Nanak=pd.DataFrame(adding_Nanak)


df_holidays=pd.concat([df_holidays,adding_Vaisakhi,adding_Nanak])

In [7]:
df_holidays.to_csv('Holy Days_2024_2030.csv',index=False)